# Testing crawler and site structure

In [1]:
import requests
import re

import urllib.parse as urlparse

from bs4 import BeautifulSoup as bs

URL_MAIN = 'https://www.inform.kz'
URL_ARCHIVE = 'https://www.inform.kz/ru/archive'

session = requests.Session()


## Check for the crawling links

### Check for article links in the first page of archives

In [3]:
url_links = 'https://www.inform.kz/ru/archive?date=01.01.2012'
response = session.get(url_links)
soup = bs(response.content, 'html.parser')

links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from main page:\n{links[:5]}...')

Links for articles (total 20 links) from main page:
['https://www.inform.kz/ru/prezident-francii-prizval-poddannyh-ne-ustraivat-isteriku_a2430381', 'https://www.inform.kz/ru/zapadnoe-polusharie-zemli-vstrechaet-novyy-god_a2430393', 'https://www.inform.kz/ru/eksperty-nazvali-10-riskov-dlya-global-noy-ekonomiki-v-2012-godu_a2430394', 'https://www.inform.kz/ru/v-vko-pozhar-unes-zhizni-pozhilogo-muzhchiny-i-2-letnego-rebenka-esche-dvoe-malyshey-ugoreli_a2430367', 'https://www.inform.kz/ru/kitayskie-analitiki-predskazyvayut-obostrenie-geopoliticheskoy-napryazhennosti-v-2012g_a2430369']...


### Check for pagination links

In [4]:
soup = bs(response.content, 'html.parser')
links = soup.find('p', class_='pagination').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Pagination links: {links}')

Pagination links: ['https://www.inform.kz/ru/archive/2?date=01.01.2012']


Check for articles from next page:

In [6]:
response = session.get(links[0])
soup = bs(response.content, 'html.parser')
links = soup.find('div', class_='news-list__col').find_all('a')
links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
links = list(links)
print(f'Links for articles (total {len(links)} links) from next page:\n{links[:5]}...')

Links for articles (total 2 links) from next page:
['https://www.inform.kz/ru/ya-ot-vsego-serdca-zhelayu-chtoby-v-novom-godu-sbylis-vse-svetlye-mechty-i-nadezhdy-kazhdogo-kazahstanca-pozdravlenie-prezidenta-rk_a2430289', 'https://www.inform.kz/ru/chislo-zhertv-ciklona-v-indii-vozroslo-do-42_a2430340']...


## Check for crawling articles

In [14]:
def check_articles(url):
    response = session.get(url)

    soup = bs(response.content, 'html.parser')

    title = soup.find('div', class_='title_article_bl')
    title = re.sub(r'\s+', ' ', title.get_text().strip())

    date = soup.find('div', class_='time_article_bl')
    date = re.sub(r'\s+', ' ', date.get_text().strip())
    
    links = []
    links_frame = soup.find('div', class_='frame_news_article_adapt')
    if links_frame:
        links = links_frame.find_all('a')
        links = set([urlparse.urlparse(URL_MAIN + link['href'].strip()).geturl() for link in links])
        links = list(links)
        links_frame.decompose()
    
    body = soup.find('div', class_='body_article_bl')
    body = re.sub(r'\s+', ' ', body.get_text().strip())
    
    tags = soup.find('div', class_='keywords_bl')
    if tags:
        tags = tags.find_all('a')
        tags = [re.sub(r'\s+', ' ', t.get_text().strip()) for t in tags]
    else:
        tags = []
    
    author = soup.find('div', class_='data_author_bl')
    if author:
        author = author.find('a')
        author = re.sub(r'\s+', ' ', author.get_text().strip())
    else:
        author = None
    
    print(f'Title: {title}')
    print(f'Date: {date}')
    print(f'Links (number of links - {len(links)}): {links[:3]}')
    print(f'Tags: {tags}')
    print(f'Author: {author}')
    print(f'Body:\n{body}')

In [15]:
url1 = 'https://www.inform.kz/ru/v-zhanaozene-prodolzhaetsya-rabota-po-blagoustroystvu-goroda_a2430452'
url2 = 'https://www.inform.kz/ru/prezident-kazahstana-provel-peregovory-s-general-nym-sekretarem-oon_a3981340'

In [16]:
check_articles(url1)

Title: В Жанаозене продолжается работа по благоустройству города
Date: 2 Января 2012 13:35
Links (number of links - 0): []
Tags: []
Author: Дамир Байманов
Body:
АСТАНА. 2 января. КАЗИНФОРМ - В г. Жанаозен продолжаются мероприятия по вывозу бытового мусора и благоустройству города, сообщает pm.kz со ссылкой на пресс-службу акимата Мангыстауской областиПо информации пресс-службы, работа по очистке города завершена. В то же время продолжается уборка бытового мусора и благоустройство города. По данным на 1 января, в Жанаозене на уборочных работах с ГКП «Тазалык» задействовано 96 человек и 23 единицы специальной техники, вывезено 1154 тонн мусора.


In [17]:
check_articles(url2)

Title: Президент Казахстана провел переговоры с Генеральным секретарем ООН
Date: 21 Сентября 2022 05:06
Links (number of links - 3): ['https://www.inform.kz/ru/glava-gosudarstva-obratilsya-k-narodu-kazahstana_a3981670', 'https://www.inform.kz/ru/glava-gosudarstva-obratilsya-k-narodu-kazahstana_a3981672', 'https://www.inform.kz/ru/glava-gosudarstva-garantiroval-spravedlivye-vybory-v-kazahstane_a3981688']
Tags: ['ООН', 'Президент Казахстана']
Author: Ринат Дусумов
Body:
НЬЮ-ЙОРК. КАЗИНФОРМ - Программа рабочего визита Касым-Жомарта Токаева в Нью-Йорк для участия в 77-й сессии Генеральной Ассамблеи ООН завершилась переговорами с Генеральным секретарем ООН Антониу Гутерришем, передает МИА «Казинформ» со ссылкой на пресс-службу Акорды. В ходе встречи, прошедшей в конструктивной и дружественной атмосфере, стороны обсудили состояние и перспективы укрепления многостороннего сотрудничества между Казахстаном и ООН. Особое внимание было уделено усилиям, направленным на выработку коллективных подхо